In [1]:
import logging
import math
import random
import statistics

import numpy as np
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler()]
)

In [3]:
SIMULATIONS = 10000
SCORING_DICT = {
    "player_pass_tds": 4,
    "player_pass_interceptions": -2,
    "player_rush_or_rec_tds": 6,
    "player_reception_yds": 0.1,
    "player_rush_yds": 0.1,
    "player_receptions": 1,
    "player_pass_yds": 0.04
}

REQUIRED_CATEGORIES_DICT = {
    'QB':['player_pass_tds','player_pass_interceptions','player_pass_yds',],
    'RB':['player_rush_or_rec_tds','player_rush_yds',],
    'WR':['player_rush_or_rec_tds','player_reception_yds','player_receptions',],
}

PPR_TYPES = ['full','half','no']

In [4]:
def create_connection():
    connection = psycopg2.connect(
    host="aws-0-us-west-1.pooler.supabase.com",
    database="postgres",
    user="postgres.bazfvnkvwteendgaxumt",
    password='?ya21[Uta=T9!w]+e]4t',
    port=6543
    )
    return connection

def import_data_supabase(supa_table):
    try:
        connection = create_connection()
        df_prior = pd.read_sql(f'SELECT * FROM {supa_table} ORDER BY player,bet_category,point',connection)
        

        print(f'{len(df_prior)} rows pulled from {supa_table}')
        return df_prior

    except Exception as error:
        raise RuntimeError(f"Error while pulling prior data: {error}.")

    finally:
        # Close the cursor and connection
        connection.close()

In [5]:
def create_mc_players(df_odds):
    players_set = set(df_odds['player'])
    mc_players = []
    for player in players_set:
        player_mask = df_odds['player'] == player
        avail_categories = set(df_odds.loc[player_mask,'bet_category'])
        for req_categories in REQUIRED_CATEGORIES_DICT.values():
            if all(rc in avail_categories for rc in req_categories):
                mc_players.append(player)
                break
    return mc_players

def create_pct_dict(df_odds,mc_player):
    pct_dict = {}
    player_mask = df_odds['player'] == mc_player
    for _,row in df_odds[player_mask].iterrows():
        cat = row['bet_category']
        pt = math.ceil(row['point'])
        pct = row['pct_under']
        
        cur_cat = pct_dict.get(cat,[])
        cur_cat.append((pt,pct))
        pct_dict[cat] = cur_cat
        
    return pct_dict

def calc_pts(cat,pct_dict):
    if not (pts_list:=pct_dict.get(cat)):
        return 0
    for pts,pct in pts_list:
        if random.random() >= pct:
            return pts
    return 0

In [6]:
def create_df_mc(mc_player,pct_dict):
    cat_cols = [col for col in SCORING_DICT.keys()]
    mc_cols = ['player'] + cat_cols
    df_mc = pd.DataFrame(columns=mc_cols)
    df_mc = df_mc.reindex(range(SIMULATIONS))
    df_mc.loc[:,'player'] = mc_player
    for col in cat_cols:
        df_mc[col] = df_mc.apply(lambda x:calc_pts(col,pct_dict),axis=1)

    # populate fantasy pts
    for i,row in df_mc.iterrows():
        total_score_ppr = 0
        for cat in cat_cols:
            pts = row[cat]
            total_score_ppr += SCORING_DICT[cat] * pts

        rec = row['player_receptions']
        df_mc.at[i,'total_score_full_ppr'] = total_score_ppr
        df_mc.at[i,'total_score_half_ppr'] = total_score_ppr - rec * 0.5
        df_mc.at[i,'total_score_no_ppr'] = total_score_ppr - rec 
    return df_mc

def create_chart_source(df_mc,ppr_type):       
    max_score = 50
    pts_dict =  {i: 0 for i in range(max_score + 1)}

    for _,row in df_mc.iterrows():
        total_score = int(round(row[f'total_score_{ppr_type}_ppr']))
        pts_dict[total_score] += 1

    for score in range(max_score + 1):
        pts_dict[score] = round(pts_dict[score]/SIMULATIONS * 100,1)

    chart_source = [{'pts':k,'pct':v} for k,v in pts_dict.items()]
    return chart_source

def create_single_row_dict(df_mc):
    single_row_dict = {}
    for col in df_mc.columns:
        if col == 'player':
            single_row_dict['player'] = df_mc[col][0]
            continue

        single_row_dict[col] = statistics.mean(df_mc[col])
        
    for ppr_type in PPR_TYPES:
        single_row_dict[f'total_score_{ppr_type}_ppr_median'] = statistics.median(df_mc[f'total_score_{ppr_type}_ppr'])
    
    for ppr_type in PPR_TYPES:
        single_row_dict[f'chart_source_{ppr_type}_ppr'] = create_chart_source(df_mc,ppr_type)
        
    return single_row_dict

In [ ]:
def create_connection():
    connection = psycopg2.connect(
    host="aws-0-us-west-1.pooler.supabase.com",
    database="postgres",
    user="postgres.bazfvnkvwteendgaxumt",
    password='?ya21[Uta=T9!w]+e]4t',
    port=6543
    )
    return connection

def export_data_supabase(df_export,supa_table):
    df_export = df_export.replace({np.nan: None})  # supabase doesn't accept NaN
    for col in df_export.columns:
        sample_val = df_export[col][0]
        if isinstance(sample_val,list):
            df_export[col] = df_export.apply(lambda x:str(x[col]).replace("'",'"'),axis=1)
    
    values = [tuple(x) for x in df_export.to_numpy()]
    insert_statement = f"INSERT INTO {supa_table} ({', '.join(df_export.columns)}) VALUES %s"

    try:
        connection = create_connection()
        cursor = connection.cursor()
        cursor.execute(f'DELETE FROM {supa_table};')
        connection.commit()

        print(f"{supa_table} deleted successfully.")

        psycopg2.extras.execute_values(cursor, insert_statement, values)
        connection.commit()

        print(f"Data inserted {len(values)} rows into {supa_table} successfully.")

    except Exception as error:
        print(f"Error while inserting data: {error}. Commits rolled back.")
        connection.rollback()

    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

In [8]:
#main
df_odds = import_data_supabase('public.full_odds')
mc_players = create_mc_players(df_odds)

data = []
logging.info('creating DataFrame...')
for mc_player in mc_players:
    pct_dict = create_pct_dict(df_odds,mc_player)
    df_mc = create_df_mc(mc_player,pct_dict)
    single_row_dict = create_single_row_dict(df_mc)
    data.append(single_row_dict)

    df_full_results = pd.DataFrame(data)
logging.info('DataFrame created, exporting to supabase...')
export_data_supabase(df_full_results,'public.chart_source')
logging.info('all done.. hooray!')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2024-10-02 19:05:53 - INFO - creating DataFrame...


2748 rows pulled from public.full_odds


2024-10-02 19:09:32 - INFO - DataFrame created, exporting to supabase...


public.chart_source deleted successfully.


2024-10-02 19:09:33 - INFO - all done.. hooray!


Data inserted 64 rows into public.chart_source successfully.


In [ ]:
# players list (id, name, team, position)
players = [
    {"id":"patrick-mahomes","name":"Patrick Mahomes","team":"KC","position":"QB"},
    {"id":"christian-mccaffrey","name":"Christian McCaffrey","team":"SF","position":"RB"},
    # ...generate this from your data
]

# projections keyed by the SAME id
projections = [
    {"id":"patrick-mahomes","ppr":22.1,"median":21,"ceiling":35,"floor":12},
    {"id":"christian-mccaffrey","ppr":24.8,"median":23,"ceiling":40,"floor":10.5},
    # ...your Monte Carlo outputs here
]

import json, os, pathlib
out = pathlib.Path("../ff_react/public")
out.mkdir(parents=True, exist_ok=True)
(out/"players.json").write_text(json.dumps(players, indent=2))
(out/"projections.json").write_text(json.dumps(projections, indent=2))
print("Wrote to ff_react/public/players.json and projections.json")
